# Day 8
## Part 1

In [19]:
type Direction = | Left | Right

let parseDirections (s: string) : Direction list =
    s 
    |> Seq.map (function
        | 'L' -> Left
        | 'R' -> Right
        | _ -> failwith "Invalid direction"
    ) |> Seq.toList

type Node = { LeftName : string; RightName : string }

let parseNode (s: string) (nodes: Map<string, Node>) =
    let parts = s.Split(Seq.toArray " =(),", StringSplitOptions.RemoveEmptyEntries)
    nodes
    |> Map.add parts[0] { LeftName = parts[1]; RightName = parts[2] }

let parseInput parseDirections parseNode inputLines =
    let directions = parseDirections (Array.head inputLines)
    let nodes =
        inputLines
        |> Array.skip 2
        |> Array.fold (fun nodes line -> parseNode line nodes) Map.empty
    (directions, nodes)

let followDirection nodes currentNodeName direction =
    let node = nodes |> Map.find currentNodeName
    match direction with
    | Left -> node.LeftName
    | Right -> node.RightName

let repeat xs = 
    seq {
        while true do yield! xs
    }

let findEndpoint nodes startName endName directions =
    repeat directions
    |> Seq.scan (followDirection nodes) startName
    |> Seq.takeWhile (fun name -> name <> endName)

let countSteps findEnd =
    findEnd
    |> Seq.length // Not + 1 for the end position because this includes the starting position


In [20]:
#r "nuget: FsUnit.xUnit"

open FsUnitTyped

let testData = 
    [|
        "LLR"
        ""
        "AAA = (BBB, BBB)"
        "BBB = (AAA, ZZZ)"
        "ZZZ = (ZZZ, ZZZ)"
    |]

let testResult =
    let (directions, nodes) = parseInput parseDirections parseNode testData
    findEndpoint nodes "AAA" "ZZZ" directions
    |> countSteps

testResult |> shouldEqual 6

Installed Packages FsUnit.xUnit, 5.6.1

In [21]:
open System.IO
let input = File.ReadAllLines "input_08.txt"

let result1 =
    let (directions, nodes) = parseInput parseDirections parseNode input
    findEndpoint nodes "AAA" "ZZZ" directions
    |> countSteps
printfn "Result 1 : %d" result1

Result 1 : 12169
